In [1]:
from BasisExpansion import BasisExpansion
import numpy as np
from LinClass import LinClass
from scipy.special import expit

In [2]:
heartData = np.loadtxt('cleanedSAheart.data.txt', skiprows=1)[:, 1:]
arrayY = heartData[:,-1]
y = np.zeros((len(heartData), 2))
for ii in range(len(heartData)):
    if arrayY[ii] == 1:
        y[ii] = np.array([1, 0])
    elif arrayY[ii] == 0:
        y[ii] = np.array([0, 1])
    else:
        print('something wrong')
        print(ii)
        print(y[ii])
X = heartData[:,0:-1]

In [3]:
Expander = BasisExpansion()
XTrain, XTest, yTrain, yTest = X[:int(len(X)/3)], X[int(len(X)/3):], y[:int(len(y)/3)], y[int(len(y)/3):]
expandedXTrain = np.zeros((len(XTrain), 4*(len(XTrain[0]) - 1) + 1))
expandedXTest = np.zeros((len(XTest), 4*(len(XTest[0])- 1) + 1))

In [4]:
index = 0
for ii in range(len(XTrain[0])):
    expandedXTrain[:,index] = XTrain[:,ii]
    expandedXTest[:,index] = XTest[:, ii]
    index +=1
    if ii == 4:
        continue
    else:
        col = XTrain[:,ii]
        knots = Expander.getKnots(5, col)
        for jj in range(len(knots[:-2])):
            expandedXTrain[:,index] = Expander.naturalCubeSplines(XTrain[:,ii], knots, jj)
            expandedXTest[:,index] = Expander.naturalCubeSplines(XTest[:,ii], knots, jj)
            index +=1

In [5]:
heartClass = LinClass(expandedXTrain, yTrain, indicatorArray = ['no heart attack', 'heart attack'])

In [6]:
print(expandedXTest.shape)
print(expandedXTrain.shape)

(308, 33)
(154, 33)


In [7]:
logisticFit = heartClass.binaryLogisticRegg()
print(logisticFit)
XtestStandardized = heartClass.standardizeTest(expandedXTest)
correct = 0
total = 0
for ii in range(len(XtestStandardized)):
    x = XtestStandardized[ii]
    probability = 1 - expit(-np.dot(x, logisticFit))
    if probability > 1/2:
        result = np.array([1, 0])
    else:
        result = np.array([0, 1])
    if (result == yTest[ii]).all():
        correct+=1 
    total +=1
print(correct/total)

91.4131246872698
18.117497925746054
4.532349240856676
0.9570104714773043
0.5279031944644477
0.2884119217212637
0.2205408503881356
0.16953153536079651
0.14933398659527522
0.1315158475756404
0.1157725005743745
0.10186231651475579
0.08957848612306782
0.07873835155173237
0.06917908934154662
0.06075524705886942
0.05333690316556565
0.046808113177060726
0.04106553333140575
0.03601717722721086
0.03158128289087267
0.027685277091654324
0.024264828351677804
0.021262982391083735
0.018629374842963814
0.016319516507896627
0.014294146540269738
0.0125186489533116
0.010962527826241079
0.009598936623107061
0.0084042571188655
0.007357723583571115
0.00644108806004965
0.005638322812154161
0.004935356262394444
0.004319839011879756
0.003780936802093028
0.0033091475409504846
0.0028961397806827717
0.002534610268506553
0.002218158437027731
0.001941175901981063
0.00169874924649118
0.0014865745467084952
0.0013008822635062738
0.0011383712758869084
0.0009961509701581538
0.0008716904200893736
0.0007627738063918087
0

In [8]:
correct = 0
total = 0
XTrainStandardized = heartClass.standardizeTest(expandedXTrain)
for ii in range(len(XTrainStandardized)):
    x = XTrainStandardized[ii]
    probability = 1 - expit(-np.dot(x, logisticFit))
    if probability > 1/2:
        result = np.array([1, 0])
    else:
        result = np.array([0, 1])
    if (result == yTrain[ii]).all():
        correct+=1 
    total +=1
print(correct/total)

0.7532467532467533
